In [1]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, StronglyConvexFunction, ConvexFunction
from PEPit.primitive_steps import proximal_step

import numpy as np
import cvxpy as cp

In [2]:
# verification mode: PEP
problem = PEPit.PEP()

rho = 1

L_smooth = 1; mu = 0.00001
f = problem.declare_function(StronglyConvexFunction, mu=mu)
g = problem.declare_function(StronglyConvexFunction, mu=mu)
# f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
# g = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
# f = problem.declare_function(ConvexFunction)
# g = problem.declare_function(ConvexFunction)

x_star, y_star, _ = (f + g).stationary_point(return_gradient_and_function_value=True)
y0 = problem.set_initial_point()

x1, grad_f_1, f_1 = proximal_step( - (1/rho) * y0, f, 1/rho)
# _ = g.value(x1)
z1, grad_g_1, g_1 = proximal_step(x1  - (1/rho) * y0, g, 1/rho)
# _ = f.value(z1)
y1 = y0 + rho * (x1 - z1)

x2, _, _ = proximal_step(z1 - (1/rho) * y1, f, 1/rho)
# _ = g.value(x2)
z2, _, _ = proximal_step(x2 + (1/rho) * y1, g, 1/rho)
# _ = f.value(z2)
y2 = y1 + rho * (x2 - z2)

E_1 = (1/rho) * (y1 - y_star)**2 + rho * (z1 - x_star)**2
E_2 = (1/rho) * (y2 - y_star)**2 + rho * (z2 - x_star)**2
Delta_1 = rho * (x2 - z2)**2 + rho * (z2 - z1)**2

problem.set_performance_metric(E_2 - (E_1 - Delta_1))

diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 7x7
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (0 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
		 function 2 : Adding 6 scalar constraint(s) ...
		 function 2 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: unbounded (solver: MOSEK); optimal value: inf


UserWarning: PEPit didn't find any nontrivial worst-case guarantee. It seems that the optimal value of your problem is unbounded.